<a href="https://colab.research.google.com/github/limseo12/YOLOv8_Object_Detection/blob/main/custom_data_RSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Vo3weMCC3nS9jZ3v6tEV")
project = rf.workspace("roboflow-58fyf").project("rock-paper-scissors-sxsw")
dataset = project.version(11).download("yolov8")

In [ ]:
!cat /content/Rock-Paper-Scissors-SXSW-11/data.yaml

In [ ]:
!pip install PyYAML

In [ ]:
#경로설정 직접경로 설정을해서 학습을 시도하면 오류가난다... 기존에 있던 yaml 데이터 그대로 학습을 시도하면 오류가 나지 않는다.
import yaml

RSP_data = {'path': '/content/Rock-Paper-Scissors-SXSW-11/',
        'train': '/content/Rock-Paper-Scissors-SXSW-11/train/images',
        'test': '/content/Rock-Paper-Scissors-SXSW-11/test/images',
        'valid': '/content/Rock-Paper-Scissors-SXSW-11/valid/images',
        'names' : ['Paper','Rock','Scissors'],
        'nc' : 3 }

with open('/content/Rock-Paper-Scissors-SXSW-11/RSP_data.yaml', 'w') as f:
  yaml.dump(data, f)

with open('/content/Rock-Paper-Scissors-SXSW-11/RSP_data.yaml', 'r') as f:
  RSP_data = yaml.safe_load(f)
  display(RSP_data)

In [ ]:
pwd

In [ ]:
%cd /content/Rock-Paper-Scissors-SXSW-11

In [ ]:
pwd

In [ ]:
!cat /content/Rock-Paper-Scissors-SXSW-11/RSP_data.yaml

## YOLOv8 설치

In [ ]:
!pip install ultralytics

In [ ]:
import ultralytics

ultralytics.checks()

## Train

In [ ]:
#Load a pre-trained model
from ultralytics import YOLO
model_x = YOLO('yolov8x.pt')

In [ ]:
#기존의 coco dataset으로 학습된 모델이란 것을 확인 할 수 있다.
print(type(model_x.names), len(model_x.names))
print(model_x.names)

In [ ]:
model_x.train(data= '/content/Rock-Paper-Scissors-SXSW-11/data.yaml', epochs=5, patience=30, batch=32, imgsz=316)

In [ ]:
results = model_x.predict(source='/content/Rock-Paper-Scissors-SXSW-11/test/images', save = True)

In [ ]:
#경로 때문에 content/runs/detect 가 아닌 content/Rock-Paper-Scissors.../runs/detect에 저장됨
print(type(results), len(results))

In [ ]:
import numpy as np

for result in results:

    uniq, cnt = np.unique(result.boxes.cls.cpu().numpy(), return_counts=True)  # Torch.Tensor -> numpy
    uniq_cnt_dict = dict(zip(uniq, cnt))

    print('\n{class num:counts} =', uniq_cnt_dict,'\n')
    for c in result.boxes.cls:
        print('class num =', int(c), ', class_name =', model_x.names[int(c)])

In [ ]:
#1번파일
from PIL import Image

with Image.open('/content/Rock-Paper-Scissors-SXSW-11/runs/detect/predict/15208484cellblock_jpg.rf.95cbda1e169a66105fbf2aa22959a73b.jpg') as pred_image:

  pred_image.show()

In [ ]:
#5번파일
from PIL import Image

with Image.open('/content/Rock-Paper-Scissors-SXSW-11/runs/detect/predict/20220216_221550_jpg.rf.02a071a383151953fcf8671fc7fca3af.jpg') as pred_image:

  pred_image.show()


In [ ]:
#6번파일

from PIL import Image

with Image.open('/content/Rock-Paper-Scissors-SXSW-11/runs/detect/predict/20220216_221819_jpg.rf.295ebb583293f91f74e1700f0ab0639a.jpg') as pred_image:

  pred_image.show()


## 결과 확인 및 다운

In [ ]:
import glob

detetced_image_list = glob.glob(('/content/Rock-Paper-Scissors-SXSW-11/runs/detect/predict'))

detected_image_nums = len(detetced_image_list)

print(detected_image_nums)

print(detetced_image_list)

In [ ]:
# 다운로드를 위한 inference image 압축

import zipfile
import os

if not os.path.exists('/content/detected_result/'):
    os.mkdir('/content/detected_result/')
    print('detected_result dir is created !!!')


with zipfile.ZipFile('/content/detected_result/detected_images.zip', 'w') as detected_images:

    for idx in range(detected_image_nums):
        detected_images.write(detetced_image_list[idx])